In [ ]:
import csv,json,time
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime
import numpy as np
import lxml, html5lib
from tqdm import tqdm

#install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from requests import Request, Session
import requests_cache, re
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import subprocess #to extract from clipboard


In [ ]:

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('headless')
cmc_urls=[]

for j in range(1,3):
    print(j)
    url = "https://coinmarketcap.com/?page="+str(j)

    wd = webdriver.Chrome(options=options)
    wd.get(url)
    soup = bs(wd.page_source, 'lxml')
    tables = soup.find_all('table')

    rows = soup.find_all("tr")
    print(rows)
    for i in range(1,len(rows)):#ignore first row which is title, and last row
        print(i)
        try:
            links = rows[i].find("a").get("href") # the first link of each row is the coingecko token url that I want to extract
            print(links)
        except:
            links=None
            print("fake row")

        if links!=None:
            #link = links[0]
            cmc_urls.append(links)

all_tokens = pd.DataFrame(cmc_urls,columns=["cmc_url"])
print(all_tokens[all_tokens.duplicated()])
all_tokens.to_csv("tokens_cmc_2024.csv", index=False)




In [ ]:
cmc_all = pd.read_csv("tokens_cmc_2024.csv")

all_url = cmc_all["cmc_url"].tolist()

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('headless')


for i in range(2): #len(all_url)
    print(i)
    url = "https://coinmarketcap.com" + all_url[i] + "historical-data/"

    wd = webdriver.Chrome(options=options)
    wd.get(url)
    time.sleep(1)

    # #click "calendar" button
    calendar_xpath = '/html/body/div[1]/div[2]/div/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]'
    #calendar_class="sc-f70bb44c-0.iQEJet.BaseButton_labelWrapper__wzpX7"
    try:
        WebDriverWait(wd,10).until(EC.presence_of_element_located((By.XPATH,calendar_xpath)))
        #WebDriverWait(wd,10).until(EC.presence_of_element_located((By.CLASS_NAME, calendar_class)))
    except:
        print("Cannot find 'calendar' button!")


    calendar_button = wd.find_element(By.XPATH,calendar_xpath)
    #calendar_button = wd.find_element(By.CLASS_NAME, calendar_class)
    wd.execute_script("arguments[0].click();", calendar_button)
    print('clicked calendar button')

    time.sleep(1)

    ## select start date: Jan 1, 2015
    # click on the header twice
    header_class = "react-datepicker__header.react-datepicker__header--custom"
    header = wd.find_element(By.CLASS_NAME, header_class)

    #header is '< March 2021 >', we want to click on 'March 2021' only, which is the second span.
    spans = header.find_elements(By.TAG_NAME, 'span')
    wd.execute_script("arguments[0].click();", spans[1])
    time.sleep(1)
    wd.execute_script("arguments[0].click();", spans[1])
    time.sleep(1)

    leftClick = wd.find_element(By.CLASS_NAME,'icon-Chevron-left')
    wd.execute_script("arguments[0].click();", leftClick)

    #select year
    year = wd.find_element(By.XPATH, "//span[text()='2015']")
    wd.execute_script("arguments[0].click();", year) 
    time.sleep(1)

    # #select month
    month = wd.find_element(By.XPATH, "//span[text()='Jan']")
    wd.execute_script("arguments[0].click();", month) 
    time.sleep(1)

    #select day
    date_class = "react-datepicker__day.react-datepicker__day--001"
    date = wd.find_element(By.CLASS_NAME, date_class)
    wd.execute_script("arguments[0].click();", date)
    time.sleep(1)

    # select end date: March 2, 2021
    # click on the header again: 

    header_class = "react-datepicker__header.react-datepicker__header--custom"
    header = wd.find_element(By.CLASS_NAME, header_class)
    spans = header.find_elements(By.TAG_NAME, 'span')
    wd.execute_script("arguments[0].click();", spans[1])
    time.sleep(1)
    wd.execute_script("arguments[0].click();", spans[1])
    time.sleep(1)

    rightClick = wd.find_element(By.CLASS_NAME,"icon-Chevron-right")
    wd.execute_script("arguments[0].click();", rightClick)
    time.sleep(1)

    year2 = wd.find_element(By.XPATH, "//span[text()='2021']")
    wd.execute_script("arguments[0].click();", year2) 
    time.sleep(1)

    month2 = wd.find_element(By.XPATH, "//span[text()='Mar']")
    wd.execute_script("arguments[0].click();", month2) 
    time.sleep(1)

    date_class2 = "react-datepicker__day.react-datepicker__day--002"
    date2 = wd.find_element(By.CLASS_NAME, date_class2)
    wd.execute_script("arguments[0].click();", date2)
    time.sleep(1)

    #click the "Continue" button
    continue_button = wd.find_element(By.XPATH, "//button[text()='Continue']")
    wd.execute_script("arguments[0].click();", continue_button)

    time.sleep(3)

    # click the 'Load More" button for as many times as needed, until it disappears
    try:
        while True:
            WebDriverWait(wd,10).until(EC.presence_of_element_located((By.XPATH,"//button[text()='Load More']")))
            load_more = wd.find_element(By.XPATH, "//button[text()='Load More']")
            wd.execute_script("arguments[0].click();", load_more)
         
    except (TimeoutException):
        print("loaded all")


    soup = bs(wd.page_source, 'lxml')
    tables = soup.find_all('table')

    df = pd.read_html(str(tables))[0] #extract entire table
    df['url'] = all_url[i]
    df.head(5)

    if i == 0:
        cmc_market = df
    else:
        cmc_market = cmc_market.append(df, ignore_index=True)

    wd.close()

cmc_market.to_csv("cmc_market_data.csv")
print('saved cmc_market_data.csv')






In [ ]:
#make all column names lower case, without special characters

cmc_market = pd.read_csv("cmc_market_data.csv")

cmc_market.rename(columns = {'Date':'date','Close**': 'close', 'Open*': 'open', 'High':'high', 'Low':'low', 'Volume':'volume', 'Market Cap': 'market_cap'}, inplace = True)
cmc_market.drop(columns = {'Unnamed: 0'}, inplace = True)

# replace values displayed as "<$0.00000001" with NaN
cmc_market = cmc_market.replace("<$0.00000001",np.nan)

#turn market variables into float
var1 = ['close', 'volume', 'market_cap']

for var in var1:
    print(var)
    cmc_market[var] = cmc_market[var].str.replace(',', '')
    cmc_market[var] = cmc_market[var].str.replace('$', '')
    cmc_market[var] = cmc_market[var].astype('float')

#change string dates to datetime
cmc_market['date'] = cmc_market['date'].str.replace(',', '')
cmc_market['date'] = cmc_market['date'].str.replace(' ', '')
cmc_market['date'] = pd.to_datetime(cmc_market['date'],format="%b%d%Y")


In [ ]:
cmc_market.to_csv("cmc_market_data.csv", index=False)





In [ ]:
category_page = "https://coinmarketcap.com/cryptocurrency-category/"

options = webdriver.ChromeOptions()
options.add_argument('--incognito')

wd = webdriver.Chrome(options=options)
wd.get(category_page)
soup = bs(wd.page_source, 'lxml')
tables = soup.find_all('table')

categories = pd.read_html(str(tables))[0] #extract entire table

category_url = []
rows = soup.find_all("tr")
for i in range(1,len(rows)):
    links = rows[i].find("a").get("href")
    category_url.append(links)

categories["category_url"] = category_url
categories = categories.drop(["Top Gainers", "Volume","#"], axis=1)
cols = categories.columns.tolist()
cols = cols[-1:]+cols[:-1] #reorder so url i smoved from last to first
categories = categories[cols]

categories.to_csv("categories.csv", index=True)




In [ ]:
#   tags (categories)
#   contracts (indicate whether there is ethereum contract)


cmc_all = pd.read_csv("tokens_cmc_2024.csv")
all_url = cmc_all["cmc_url"].tolist()

cmc_all['main contract'] = np.nan
cmc_all['other contracts'] = np.nan
cmc_all['tags_text'] = np.nan

showAll_class = 'sc-f70bb44c-0.sc-9ee74f67-1.ixMiII'
tag_class = "sc-f70bb44c-0.sc-9ee74f67-1.dWtIZr"
chain_class = "chain-name"

options = webdriver.ChromeOptions()
options.add_argument('--incognito')


for k in range(1,3):#len(all_url)
    
    print(k)
    url = "https://coinmarketcap.com"+all_url[k] 
    print(url)

    wd = webdriver.Chrome(options=options)
    wd.get(url)

    ## Scrape Contracts
    ## Get link to each token's contract on its main chain, 
    # eg. https://arbiscan.io/token/0x912CE59144191C1204E64559FE8253a0e49E6548 for ARB

    try:
        WebDriverWait(wd,10).until(EC.presence_of_element_located((By.CLASS_NAME, chain_class)))
        main_chain_link = wd.find_element(By.CLASS_NAME, chain_class)
        print('found main contract')
        link = main_chain_link.get_attribute('href')

        #Save the main contract in the cmc_all table for each token
        # get row index of the token:
        row = cmc_all.index[cmc_all['cmc_url']==all_url[k]][0]
        # insert contract for the token:
        cmc_all['main contract'][row] = link


    except:      
        print("can't find main contract")


    ## Get links to each token's contracts on the other chains / L2s

    # Click the "More" button
    more_xpath = "/html/body/div[1]/div[2]/div/div[2]/div/div/div[2]/div[2]/section[2]/div/div[2]/div[1]/div[2]/div/div[2]"

    try:
        WebDriverWait(wd,10).until(EC.presence_of_element_located((By.XPATH, more_xpath)))
        print('found more button')

        more_button = wd.find_element(By.XPATH, more_xpath)
        chain = ActionChains(wd)
        chain.move_to_element(more_button).perform()

        time.sleep(3)

        chains_link = wd.find_elements(By.CLASS_NAME, chain_class)
        more_links = [i.get_attribute('href') for i in chains_link]

        #Save the other contracts in the cmc_all table for each token
        # get row index of the token:
        row = cmc_all.index[cmc_all['cmc_url']==all_url[k]][0]
        # insert contract for the token:
        cmc_all['other contracts'][row] = more_links

    except:      
        print("can't find more button")

    ## Scrape Tags
    try:
    # If the Show all button is available, click on it and scrape all the tags

        WebDriverWait(wd,10).until(EC.presence_of_element_located((By.CLASS_NAME, showAll_class)))
        print('found Show all button')

        showAll_button = wd.find_element(By.CLASS_NAME, showAll_class)
        wd.execute_script("arguments[0].click();", showAll_button)
        time.sleep(1)

        tag_link = wd.find_elements(By.CLASS_NAME, tag_class)
        tags_text = [i.text for i in tag_link]

        #Save tags in the cmc_all table for each token
        # get row index of the token:
        row = cmc_all.index[cmc_all['cmc_url']==all_url[k]][0]
        # insert tags for the token:
        cmc_all['tags_text'][row] = tags_text

    except:      
        print("can't find Show all button")
        # If Show all button is unavailable, just scrape the tags shown
        try:
            WebDriverWait(wd,10).until(EC.presence_of_element_located((By.CLASS_NAME, tag_class)))
            print('found Tags')
            tag_link = wd.find_elements(By.CLASS_NAME, tag_class)
            tags_text = [i.text for i in tag_link]

            #Save tags in the cmc_all table for each token
            # get row index of the token:
            row = cmc_all.index[cmc_all['cmc_url']==all_url[k]][0]
            # insert tags for the token:
            cmc_all['tags_text'][row] = tags_text
        except:
            print('No tags')


    wd.close()



In [ ]:

url = "https://coinmarketcap.com/currencies/sperax/"

options = webdriver.ChromeOptions()
options.add_argument('--incognito')

wd = webdriver.Chrome(options=options)
wd.get(url)

cmc_all['tags_link'] = np.nan
cmc_all['tags_text'] = np.nan


showAll_class = 'sc-f70bb44c-0.sc-9ee74f67-1.ixMiII'
tag_class = "sc-f70bb44c-0.sc-9ee74f67-1.dWtIZr"


try:
    # If the Show all button is available, click on it and scrape all the tags

    WebDriverWait(wd,10).until(EC.presence_of_element_located((By.CLASS_NAME, showAll_class)))
    print('found Show all button')

    showAll_button = wd.find_element(By.CLASS_NAME, showAll_class)
    wd.execute_script("arguments[0].click();", showAll_button)
    time.sleep(1)

    tag_link = wd.find_elements(By.CLASS_NAME, tag_class)
    tags_text = [i.text for i in tag_link]

    #Save tags in the cmc_all table for each token
    # get row index of the token:
    row = cmc_all.index[cmc_all['cmc_url']==all_url[k]][0]
    # insert tags for the token:
    cmc_all['tags_text'][row] = tags_text

except:      
    print("can't find Show all button")
    # If Show all button is unavailable, just scrape the tags shown
    try:
        WebDriverWait(wd,10).until(EC.presence_of_element_located((By.CLASS_NAME, tag_class)))
        print('found Tags')
        tag_link = wd.find_elements(By.CLASS_NAME, tag_class)
        tags_text = [i.text for i in tag_link]

        #Save tags in the cmc_all table for each token
        # get row index of the token:
        row = cmc_all.index[cmc_all['cmc_url']==all_url[k]][0]
        # insert tags for the token:
        cmc_all['tags_text'][row] = tags_text
    except:
        print('No tags')

#<span class="sc-f70bb44c-0 sc-9ee74f67-1 dWtIZr"><a href="/view/scaling/" class="cmc-link">Scaling</a></span>

## 

In [ ]:

#drop tokens without instrinsic value: ie pegged to something else. 
cmc_all = pd.read_csv("cmc_all_tags1.csv") # should I be using cmc_all_tags.csv instead?
indexnames = cmc_all[(cmc_all['Stablecoin']==1) | (cmc_all["Synthetics"]==1)|(cmc_all["Tokenized Stock"]==1 )
                   | (cmc_all['Wrapped Tokens']==1) | (cmc_all['ETH 2.0 Staking']==1)
                   | (cmc_all['ETH 2.0 Staking']==1)].index
cmc_all.drop(indexnames,inplace=True)

#defi combines: DeFi, Yield Farming, AMM, Oracles, Lending/borrowing, derivatives, yield aggregator, insurance, rebase, seigniorage, options, Defi index,DEX, yearn partnership
cmc_all['defi'] = np.where((cmc_all["DeFi"]==1 )|(cmc_all["Yield farming"]==1) |(cmc_all["AMM"]==1)|(cmc_all["Lending / Borrowing"]==1)
                |(cmc_all["Derivatives"]==1)|(cmc_all["Yield Aggregator"]==1)|(cmc_all["Insurance"]==1)
                | (cmc_all["Rebase"]==1) |(cmc_all["Seigniorage"]==1) |(cmc_all["Options"]==1)
                | (cmc_all["DeFi Index"]==1) | (cmc_all["Decentralized exchange"]==1) | (cmc_all["Yearn Partnerships"]==1),1, np.nan)

# distributed_comp combines: Filesharing, storage, distributed computing
cmc_all["distributed_comp"] = np.where((cmc_all["Filesharing"]==1 )|(cmc_all["Storage"]==1 )|(cmc_all["Distributed Computing"]==1),1,np.nan)

#entertainment combines: Content Creation, Media, Memes, Videos, entertainment, music, fan token, 
#                       communication and social media, events, social money, gaming, sports, gambling
cmc_all["entertainment"] = np.where((cmc_all["Content Creation"]==1)|(cmc_all["Media"]==1)|(cmc_all["Memes"]==1)|
                        (cmc_all["Video"]==1)|(cmc_all["Entertainment"]==1)| (cmc_all["Music"]==1)|
                        (cmc_all["Fan token"]==1)|(cmc_all["Communications & Social Media"]==1)|(cmc_all["Events"]==1)
                        |(cmc_all["Social Money"]==1)|(cmc_all["Gaming"]==1)|(cmc_all["Sports"]==1)|(cmc_all["Gambling"]==1)|(cmc_all["NFTs & Collectibles"]==1),1,np.nan)

#infrastructure combines scaling, interoperability, smart contract platform
cmc_all["infrastructure"] = np.where((cmc_all["Scaling"]==1)|(cmc_all["Interoperability"]==1)|(cmc_all["Smart Contracts"]==1),1,np.nan)

cmc_all = cmc_all.rename(columns = {"Centralized exchange": "cex", "Privacy":"privacy","Asset management":"asset_management"})
cmc_all["biz_solution"] = np.where((cmc_all["Identity"]==1)|(cmc_all["Analytics"]==1)|(cmc_all["Marketing"]==1)|
                            (cmc_all["Logistics"]==1),1,np.nan)

#drop categories that have been combined together. 
cmc_all = cmc_all.drop(["DeFi","Yield farming","AMM","Lending / Borrowing","Derivatives","Yield Aggregator"
                        ,"Insurance","Rebase","Seigniorage","Options","DeFi Index","Decentralized exchange"
                        ,"Yearn Partnerships","Content Creation","Media","Memes","Video","Entertainment","Music"
                        ,"Fan token","Communications & Social Media","Events","Social Money","Gaming","Sports"
                        ,"Gambling","NFTs & Collectibles","Scaling","Interoperability","Smart Contracts","Identity"
                        ,"Analytics","Marketing","Logistics"], axis=1)

In [ ]:
cmc_all.to_csv("cmc_industry_groups.csv", index=False)

In [ ]:
cmc_contract = pd.read_csv("cmc_contract_all.csv")
industries = pd.read_csv("cmc_industry_groups.csv")
cmc_market = pd.read_csv("cmc_market_all.csv")
cmc_market = cmc_market.rename(columns = {"url":"cmc_url"})
contract_industries = cmc_contract.merge(industries, how="outer", on=["cmc_url"])
market_data = contract_industries.merge(cmc_market,how="outer",on=["cmc_url"])
market_data.to_csv("market_data.csv",index=False)
